<a href="https://colab.research.google.com/github/trefftzc/partition/blob/main/Partition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The partition problem

The partition problem is intuitive: Given a collection of n objects, can we place part of those objects in the left hand of a balance and the remaining objects in the right hand side of a balance so that they weigh the same? In other (more mathematical) words, if we have a multiset of integers s, can we partition that multiset into two separate multisets q and r so that the sum of the elements in q is the same as the sum of the elements in r. A more formal statement of the problem can be found on mathworld.wolfram [here](https://mathworld.wolfram.com/NumberPartitioningProblem.html).

This simple problem belongs to a very unique kind of problems called NP-complete problems that are very hard to solve.
Given an instance of the partition problem with n positive integer values, one way to find if there is a solution to that instance is to examine each and every one of the $2^n$ subsets and to check for every one of those subsets if that subset and its complement are a solution to this instance of the partition problem. One adds up the elements in the subset, adds up the elements in the complement and if the two sums are the same, this is a solution to this instance of the partition problem.

Consider the following example: {2,3,5}. In this case n, the number of elements in the instance of the partition problem, is 3. If one calculates all the possible subsets, there are $2^3$ = 8 possible distinct subsets. The following table shows the eight subsets, their complements and indicates which of those subsets are solutions for this instance of the partition problem.

| Index | Binary Encoding | Elements | Elements in the Complement | Solution |
|:-------|:-----------------|:----------|:------------------------|:--------------|
| 0 |	000	| {}	| {2,3,5}	| No |
|1 |	001	|{2}	|{3,5}	|No
|2 |	010	|{3}	|{2,5}	|No
|3 |	011	|{2,3}	|{5}	|Yes
|4 |	100	|{5}	|{2,3}	|Yes
|5 |	101	|{2,5}	|{3}	|No
|6 |	110	|{3,5}	|{2}	|No
|7 |	111	|{2,3,5}	|{}	|No

Notice that the first half of the table is symmetrical to the second half of the table. Hence, one only needs to go through the first half of all the elements in the Power Set.



In [ ]:
#
# Program that solves the partition problem in python
# Sequential version
#
import time

def evaluatePartition(  value,  n, array) :
   sum0s = 0
   sum1s = 0
   mask = 1
   for i in range(0,n):
    if ((mask & value) != 0):
      sum1s = sum1s + array[i]
    else:
      sum0s = sum0s + array[i]
    mask = mask * 2
   if (sum0s == sum1s):
     return 1
   else:
     return 0


def printResults(value, n, array):

  print("Solution:\n")
  print("First partition: ")
  mask = 1
  sum = 0
  for i in range(0,n):
    if ((mask & value) != 0):
      print(array[i]," ")
      sum = sum + array[i]
    mask = mask * 2

  print(" sum: ",sum)
  print("Second partition: ")
  mask = 1
  sum = 0
  for i in range(0,n):
    if ((mask & value) == 0):
      print(array[i]," ")
      sum = sum + array[i]

    mask = mask * 2

  print(" sum: \n",sum)



if __name__ == "__main__":

  # n is the size of the array with the integer values
  # array contains the set of integer values

  n = 24

  print("The value of n is \n",n)
  array=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,23]


  print("The values in the multiset are: \n")
  for i in range(0,n):
    print(array[i]," ")

  print("\n")

  nPartitions = 1
  # Only half of all possible partitions need be examined
  # The second half is symmetrical to the first half
  for i in range(1,n):
    nPartitions = nPartitions * 2

  print("The number of possible partitions is: ",nPartitions)

  solutionFound = 0
  solution = -1
  start = time.time()
  for i in range(1,nPartitions):
    if (evaluatePartition( i,  n, array) == 1):
      solutionFound = 1
      solution = i
      break

  end = time.time()
  if (solutionFound):
    printResults(solution, n, array)
  else:
    printf("No solution was found.")
  print("Execution time: ",end-start)


The value of n is 
 24
The values in the multiset are: 

1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
23  


The number of possible partitions is:  8388608
Solution:

First partition: 
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
 sum:  23
Second partition: 
23  
 sum: 
 23
Execution time:  37.07309126853943


Using NUMBA, a Just In Time compiler (JIT), can make the code run faster:


In [ ]:
#
# Program that solves the partition problem in python
# Sequential version with numba
#
import sys
from numba import njit
import numpy as np
import time

@njit
def evaluatePartition(  array,value:np.dtype=np.int64,  n:np.dtype=np.int64) :
   # print("Evaluate partition called with parameter: ",value)
   sum0s = 0
   sum1s = 0
   mask = 1
   for i in range(0,n):
    if ((mask & value) != 0):
      sum1s = sum1s + array[i]
    else:
      sum0s = sum0s + array[i]
    mask = mask * 2
   if (sum0s == sum1s):
     # print("Evaluate partition ",value," returns ",value)
     return value
   else:
    # print("Evaluate partition ",value," returns ",0)
    return 0

@njit
def printResults(value, n, array):
  print("Solution:\n")
  print("First partition: ")
  mask = 1
  sum = 0
  for i in range(0,n):

    if ((mask & value) != 0):
      print(array[i]," ")
      sum = sum + array[i]
    mask = mask * 2

  print(" sum: ",sum)
  print("Second partition: ")
  mask = 1
  sum = 0
  for i in range(0,n):
    if ((mask & value) == 0):
      print(array[i]," ")
      sum = sum + array[i]

    mask = mask * 2

  print(" sum: \n",sum)

@njit(nogil=True)
def sequentialFor(n,array,nPartitions):
  solutionFound = 0
  solution = -1
  result = np.zeros(nPartitions,dtype=np.int64)

  for i in range(1,nPartitions):
    result[i] = evaluatePartition( array,i,  n)

  # print("At the end array contains: ",result)
  solutionFound = np.max(result)
  solution = solutionFound


  if (solutionFound):
    printResults(solution, n, array)
  else:
    print("No solution was found.")



if __name__ == "__main__":

  # n is the size of the array with the integer values
  # array contains the set of integer values
  # n is the size of the array with the integer values
  # array contains the set of integer values

  n = 24

  print("The value of n is \n",n)
  array=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,23]


  print("The values in the multiset are: \n")

  for i in range(0,n):
    print(array[i]," ")

  print("\n")

  nPartitions = 1
  # Only half of all possible partitions need be examined
  # The second half is symmetrical to the first half
  for i in range(1,n):
    nPartitions = nPartitions * 2

  print("The number of possible partitions is: ",nPartitions)
  npArray = np.array(array)
  start = time.time()
  sequentialFor(n,npArray,nPartitions)
  end = time.time()
  print("Execution time: ",end-start)


The value of n is 
 24
The values in the multiset are: 

1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
23  


The number of possible partitions is:  8388608
Solution:

First partition: 
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
 sum:  23
Second partition: 
23  
 sum: 
 23
Execution time:  2.262192487716675


It is possible to try to improve even futher, by parallelizing the execution of the main loop with prange instead of range.

In [ ]:
#
# Program that solves the partition problem in python
# Parallel version with numba
#
import sys
from numba import njit,prange
import numpy as np
import time

@njit
def evaluatePartition(  array,value:np.dtype=np.int64,  n:np.dtype=np.int64) :
   # print("Evaluate partition called with parameter: ",value)
   sum0s = 0
   sum1s = 0
   mask = 1
   for i in range(0,n):
    if ((mask & value) != 0):
      sum1s = sum1s + array[i]
    else:
      sum0s = sum0s + array[i]
    mask = mask * 2
   if (sum0s == sum1s):
     # print("Evaluate partition ",value," returns ",value)
     return value
   else:
    # print("Evaluate partition ",value," returns ",0)
    return 0

@njit
def printResults(value, n, array):
  print("Solution:\n")
  print("First partition: ")
  mask = 1
  sum = 0
  for i in range(0,n):

    if ((mask & value) != 0):
      print(array[i]," ")
      sum = sum + array[i]
    mask = mask * 2

  print(" sum: ",sum)
  print("Second partition: ")
  mask = 1
  sum = 0
  for i in range(0,n):
    if ((mask & value) == 0):
      print(array[i]," ")
      sum = sum + array[i]

    mask = mask * 2

  print(" sum: \n",sum)

@njit(parallel=True,nogil=True)
def parallelFor(n,array,nPartitions):
  solutionFound = 0
  solution = -1
  result = np.zeros(nPartitions,dtype=np.int64)

  for i in prange(1,nPartitions):
    result[i] = evaluatePartition( array,i,  n)

  # print("At the end array contains: ",result)
  solutionFound = np.max(result)
  solution = solutionFound


  if (solutionFound):
    printResults(solution, n, array)
  else:
    print("No solution was found.")



if __name__ == "__main__":

  # n is the size of the array with the integer values
  # array contains the set of integer values
  # n is the size of the array with the integer values
  # array contains the set of integer values

  n = 24

  print("The value of n is \n",n)
  array=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,23]


  print("The values in the multiset are: \n")
  for i in range(0,n):
    print(array[i]," ")

  print("\n")

  nPartitions = 1
  # Only half of all possible partitions need be examined
  # The second half is symmetrical to the first half
  for i in range(1,n):
    nPartitions = nPartitions * 2

  print("The number of possible partitions is: ",nPartitions)
  npArray = np.array(array)
  start = time.time()
  parallelFor(n,npArray,nPartitions)
  end = time.time()
  print("Execution time: ",end-start)


The value of n is 
 24
The values in the multiset are: 

1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
23  


The number of possible partitions is:  8388608
Solution:

First partition: 
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
1  
 sum:  23
Second partition: 
23  
 sum: 
 23
Execution time:  2.1367759704589844
